In [ ]:
###########借鉴https://blog.csdn.net/y0367/article/details/51501780
###########并运用GridSearchCV进行超参数的改进，加快运行速度（参考https://blog.csdn.net/weixin_41988628/article/details/83098130）

In [5]:
import pandas as pd # 数据科学计算工具
import numpy as np # 数值计算工具
import matplotlib.pyplot as plt # 可视化
import seaborn as sns # matplotlib的高级API
from sklearn.model_selection import StratifiedKFold #交叉验证
from sklearn.model_selection import GridSearchCV #网格搜索
from sklearn.model_selection import train_test_split #将数据集分开成训练集和测试集
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
##地址自己填
train = pd.read_csv("地址", dtype={"Age": np.float64},)

In [6]:
def harmonize_data(titanic):
    # 填充空数据 和 把string数据转成integer表示
    # 对于年龄字段发生缺失，我们用所有年龄的均值替代
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    # 性别男： 用0替代
    titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
    # 性别女： 用1替代
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 1

    titanic["Embarked"] = titanic["Embarked"].fillna("S")

    titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
    titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
    titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2

    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())

    return titanic

train_data = harmonize_data(train)

In [12]:
# 列出对生存结果有影响的字段
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
# 存放不同参数取值，以及对应的精度，每一个元素都是一个三元组(a, b, c)
results = []
estimator=RandomForestClassifier(min_samples_split=100,min_samples_leaf=20,max_depth=8,max_features='sqrt',random_state=10)
# 最小叶子结点的参数取值
min_samples_leaf = list(range(1, 50, 3))
# 决策树个数参数取值
n_estimators = list(range(1, 100, 5))
random_state = [50]

seed = 7 #重现随机生成的训练
test_size = 0.33 #33%测试，67%训练
X_train, X_test, Y_train, Y_test = train_test_split(train_data[predictors], train_data['Survived'], test_size=test_size, random_state=seed)
model = RandomForestClassifier()
param_grid = dict(min_samples_leaf = min_samples_leaf,n_estimators = n_estimators,random_state = random_state)#转化为字典格式，网络搜索要求
kflod = StratifiedKFold(n_splits=10, shuffle = True,random_state=7)#将训练/测试数据集划分10个互斥子集
grid_search = GridSearchCV(model,param_grid,scoring = 'neg_log_loss',n_jobs = -1,cv = kflod)
#scoring指定损失函数类型，n_jobs指定全部cpu跑，cv指定交叉验证
grid_result = grid_search.fit(X_train, Y_train) #运行网格搜索

print("Best: %f using %s" % (grid_result.best_score_,grid_search.best_params_))
#grid_scores_：给出不同参数情况下的评价结果。best_params_：描述了已取得最佳结果的参数的组合
#best_score_：成员提供优化过程期间观察到的最好的评分
#具有键作为列标题和值作为列的dict，可以导入到DataFrame中。
#注意，“params”键用于存储所有参数候选项的参数设置列表。
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))


Best: -0.396677 using {'min_samples_leaf': 4, 'n_estimators': 36, 'random_state': 50}
-7.768784  with:   {'min_samples_leaf': 1, 'n_estimators': 1, 'random_state': 50}
-1.800509  with:   {'min_samples_leaf': 1, 'n_estimators': 6, 'random_state': 50}
-1.361935  with:   {'min_samples_leaf': 1, 'n_estimators': 11, 'random_state': 50}
-1.254842  with:   {'min_samples_leaf': 1, 'n_estimators': 16, 'random_state': 50}
-1.039673  with:   {'min_samples_leaf': 1, 'n_estimators': 21, 'random_state': 50}
-1.041810  with:   {'min_samples_leaf': 1, 'n_estimators': 26, 'random_state': 50}
-0.940821  with:   {'min_samples_leaf': 1, 'n_estimators': 31, 'random_state': 50}
-0.835405  with:   {'min_samples_leaf': 1, 'n_estimators': 36, 'random_state': 50}
-0.786503  with:   {'min_samples_leaf': 1, 'n_estimators': 41, 'random_state': 50}
-0.789034  with:   {'min_samples_leaf': 1, 'n_estimators': 46, 'random_state': 50}
-0.790097  with:   {'min_samples_leaf': 1, 'n_estimators': 51, 'random_state': 50}
-0.

In [13]:

alg = RandomForestClassifier(min_samples_leaf=4, n_estimators=36, random_state=50)
alg.fit(X_train, Y_train)
predict = alg.predict(X_test)
# 真实结果和预测结果进行比较，计算准确率
print((Y_test == predict).mean())


0.7864406779661017
